In [3]:
import pandas as pd
df = pd.read_csv("Equity.csv")
pd.set_option("display.max_columns", 10)
pd.set_option("display.max_rows", 10)

In [4]:
df.head()

,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,...,R&D Expense Growth,SG&A Expenses Growth,Sector,2019 PRICE VAR [%],Class
0,CMCSA,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,...,0.0000,0.1308,Consumer Cyclical,32.794573,1
1,KMI,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,...,0.0000,-0.1265,Energy,40.588068,1
2,INTC,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,...,0.0390,-0.0942,Technology,30.295514,1
3,MU,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,...,0.1738,0.0942,Technology,64.213737,1
4,GE,1.220000e+11,0.0285,9.546100e+10,2.615400e+10,...,0.0000,0.0308,Industrials,44.757840,1


In [5]:
df.isnull().any()

Unnamed: 0              False
Revenue                  True
Revenue Growth           True
Cost of Revenue          True
Gross Profit             True
                        ...  
R&D Expense Growth       True
SG&A Expenses Growth     True
Sector                  False
2019 PRICE VAR [%]      False
Class                   False
Length: 225, dtype: bool

In [6]:
df = df.drop(columns=['cashConversionCycle' , 'operatingCycle'])
df.columns.sort_values()

Index(['10Y Dividend per Share Growth (per Share)',
       '10Y Net Income Growth (per Share)',
       '10Y Operating CF Growth (per Share)', '10Y Revenue Growth (per Share)',
       '10Y Shareholders Equity Growth (per Share)', '2019 PRICE VAR [%]',
       '3Y Dividend per Share Growth (per Share)',
       '3Y Net Income Growth (per Share)',
       '3Y Operating CF Growth (per Share)', '3Y Revenue Growth (per Share)',
       ...
       'priceToBookRatio', 'priceToFreeCashFlowsRatio',
       'priceToOperatingCashFlowsRatio', 'priceToSalesRatio', 'quickRatio',
       'returnOnAssets', 'returnOnCapitalEmployed', 'returnOnEquity',
       'shortTermCoverageRatios', 'totalDebtToCapitalization'],
      dtype='object', length=223)

In [7]:
## Feature Selection
c = df.corr()
c = abs(c['Class'])
print(c[c>0.1].sort_values()) ## Sort by categories where there is only more than 0.1 correlation to the class category

/var/folders/2b/m2w655kn0s93bz8jzv2nhkp80000gn/T/ipykernel_52700/1860318574.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  c = df.corr()


Net Income Com                         0.100404
Consolidated Income                    0.103065
Net Income                             0.103635
EBIT                                   0.108232
Total non-current liabilities          0.108440
                                         ...   
10Y Revenue Growth (per Share)         0.138813
Dividend per Share                     0.144091
10Y Operating CF Growth (per Share)    0.145157
2019 PRICE VAR [%]                     0.423306
Class                                  1.000000
Name: Class, Length: 16, dtype: float64


In [8]:
df = df.loc[:, ["Class", "Dividend per Share", "10Y Net Income Growth (per Share)", "10Y Shareholders Equity Growth (per Share)", "EBIT", "Net Income", "Sector"]] #df.columns

In [9]:
df.isnull().any()

Class                                         False
Dividend per Share                             True
10Y Net Income Growth (per Share)              True
10Y Shareholders Equity Growth (per Share)     True
EBIT                                           True
Net Income                                     True
Sector                                        False
dtype: bool

In [10]:
## Data Cleaning
m = df['10Y Net Income Growth (per Share)'].mean()
df['10Y Net Income Growth (per Share)'].fillna(m,inplace=True)
m = df['Dividend per Share'].mean()
df['Dividend per Share'].fillna(m,inplace=True)
m = df["10Y Shareholders Equity Growth (per Share)"].mean()
df["10Y Shareholders Equity Growth (per Share)"].fillna(m,inplace=True)
m = df["EBIT"].mean()
df["EBIT"].fillna(m,inplace=True)
m = df['Net Income'].mean()
df["Net Income"].fillna(m,inplace=True)
df = df.dropna()






In [11]:
dummy = pd.get_dummies(df['Sector'])
df = df.merge(dummy,left_index=True,right_index=True)
df = df.drop(columns="Sector")


In [12]:
df

,Class,Dividend per Share,10Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),EBIT,...,Healthcare,Industrials,Real Estate,Technology,Utilities
0,1,0.917,0.192800,0.084100,1.865300e+10,...,0,0,0,0,0
1,1,0.725,0.038281,0.031093,4.113000e+09,...,0,0,0,0,0
2,1,1.200,0.171900,0.087600,2.319100e+10,...,0,0,0,1,0
3,1,0.000,0.000000,0.133600,1.464500e+10,...,0,0,0,1,0
4,1,0.370,0.000000,-0.102600,-1.671300e+10,...,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4387,0,0.000,0.038281,0.031093,-3.766024e+06,...,0,0,1,0,0
4388,0,0.000,0.000000,-0.428800,-9.170000e+06,...,0,0,0,0,0
4389,0,0.000,0.038281,0.031093,9.655494e+06,...,0,0,0,0,0
4390,0,0.000,0.038281,0.031093,-1.664769e+07,...,0,1,0,0,0


In [13]:
## Normalizing
from scipy import stats
df['EBIT'] = stats.zscore(df['EBIT'])
df['Net Income'] = stats.zscore(df['Net Income'])

In [14]:
X = df.drop(columns = "Class")
Y = df['Class']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state= 42)
from sklearn import neural_network
model = neural_network.MLPClassifier(max_iter=10000)
model.fit(x_train, y_train)
pred = model.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred)
cm
print("Accuracy: ", (cm[0,0] + cm[1,1])/ sum(sum(cm)))


Accuracy:  0.7103825136612022


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/imp.py", line 343, in load_dynamic
    retu

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
f = SelectKBest(score_func = f_regression, k =10)
f = f.fit(X,Y)
f = f.get_support()
f = X.columns[f]
f

NameError: name 'X' is not defined

hello world
